<a href="https://colab.research.google.com/github/PrajwalRaj21/Travel_recommendation/blob/main/TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from google.colab import files

# Read the CSV file (assuming 'travel1.csv' was uploaded)
filename = "travel1.csv"
df = pd.read_csv(filename)

# Display first few rows
df.head()


,pID,pName,Latitude,Longitude,Tags,province number,province name,district,Latitude.1,Longitude.1,latitude(Rd),longitude(Rd),latitude(rp),longitude(rp)
0,1.0,Pashupatinath temple,27.71051,85.34881,"Historical, temple, statue",3.0,Bagmati,Kathmandu,27.710314,85.322163,0.483636,1.489153,0.483640,1.489618
1,2.0,Hanumandhoka palace,27.70443,85.30738,"Temple,sightseeing, historical",3.0,Bagmati,Kathmandu,27.710314,85.322163,0.483636,1.489153,0.483534,1.488895
2,3.0,Chitwan national park,27.51929,84.31353,"trekking, wildlife,lakes,river",3.0,Bagmati,Chitwan,27.529131,84.354205,0.480474,1.472259,0.480302,1.471549
3,4.0,Kalinchowk bhagwati temple,27.75781,86.03360,"Temple,sightseeing,Religious",3.0,Bagmati,Dolakha,27.778429,86.175176,0.484825,1.504041,0.484465,1.501570
4,5.0,Baudhanath stupa,27.72147,85.36204,"Temple,statues",3.0,Bagmati,Kathmandu,27.710314,85.322163,0.483636,1.489153,0.483831,1.489849


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv('travel1.csv')

# Add Approximate Budget and Number of Days
def estimate_budget_and_days(tags):
    # Convert tags to string if it's not already
    tags = str(tags)  # This line ensures tags is always a string
    if 'trekking' in tags or 'hiking' in tags:
        return 1000, 7  # Higher budget and more days for trekking/hiking
    elif 'temple' in tags or 'cultural' in tags:
        return 500, 3   # Lower budget and fewer days for temples/cultural sites
    elif 'lake' in tags or 'pond' in tags:
        return 300, 2   # Lower budget and fewer days for lakes/ponds
    else:
        return 700, 5   # Default budget and days

df['Approximate Budget'], df['Number of Days'] = zip(*df['Tags'].apply(estimate_budget_and_days))

# ... (rest of your code remains the same)

# TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')
df['Tags'] = df['Tags'].fillna('')  # Replace NaN with empty string
tfidf_matrix = tfidf.fit_transform(df['Tags'])
# Function to get recommendations based on user input
def get_recommendations(user_input, budget, days, tfidf_matrix, df, top_n=5):
    # Filter places based on budget and days
    filtered_df = df[(df['Approximate Budget'] <= budget) & (df['Number of Days'] <= days)]

    if filtered_df.empty:
        return "No places match your budget and days criteria."

    # Transform user input into TF-IDF vector
    user_input_vec = tfidf.transform([user_input])

    # Compute cosine similarity between user input and filtered places
    cosine_sim = cosine_similarity(user_input_vec, tfidf_matrix[filtered_df.index])
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top N recommendations
    top_indices = [i[0] for i in sim_scores[:top_n]]
    return filtered_df.iloc[top_indices]

# Function to generate itinerary
def generate_itinerary(recommendations, days):
    itinerary = {}
    for day in range(1, days + 1):
        if day == 1:
            # Day 1: Start with less strenuous activities (e.g., temples, cultural sites)
            day_activities = recommendations[recommendations['Tags'].str.contains('temple|cultural')]
        elif day == days:
            # Last day: End with relaxing activities (e.g., lakes, ponds)
            day_activities = recommendations[recommendations['Tags'].str.contains('lake|pond')]
        else:
            # Middle days: Include adventurous activities (e.g., trekking, hiking)
            day_activities = recommendations[recommendations['Tags'].str.contains('trekking|hiking')]

        # If no specific activities are found, pick random activities
        if day_activities.empty:
            day_activities = recommendations.sample(min(2, len(recommendations)))

        # Add activities to the itinerary for the day
        itinerary[f"Day {day}"] = day_activities[['pName', 'Tags', 'Approximate Budget', 'Number of Days']].to_dict('records')

    return itinerary

# Get user input
user_input = input("Describe the type of place you want to visit (e.g., 'I want to visit temples, lakes, and do trekking'): ")
budget = float(input("Enter your budget (e.g., 1000): "))
days = int(input("Enter the number of days you plan to stay (e.g., 5): "))

# Get recommendations
recommendations = get_recommendations(user_input, budget, days, tfidf_matrix, df)

# Generate itinerary
if isinstance(recommendations, str):
    print(recommendations)
else:
    itinerary = generate_itinerary(recommendations, days)
    for day, activities in itinerary.items():
        print(f"\n{day}:")
        for activity in activities:
            print(f"- {activity['pName']} ({activity['Tags']}) | Budget: ${activity['Approximate Budget']} | Days: {activity['Number of Days']}")

Describe the type of place you want to visit (e.g., 'I want to visit temples, lakes, and do trekking'): temples
Enter your budget (e.g., 1000): 100000
Enter the number of days you plan to stay (e.g., 5): 5

Day 1:
- Hanuman Dhoka (temples, cultural) | Budget: $500 | Days: 3
- Thimi (cultural, temples) | Budget: $500 | Days: 3
- Sanga (statues, cultural, temples) | Budget: $500 | Days: 3
- Devghat (Religious, cultural, temples,river) | Budget: $500 | Days: 3
- Patan durbar square (temples,statues,cultural,sculpture) | Budget: $500 | Days: 3

Day 2:
- Sanga (statues, cultural, temples) | Budget: $500 | Days: 3
- Thimi (cultural, temples) | Budget: $500 | Days: 3

Day 3:
- Thimi (cultural, temples) | Budget: $500 | Days: 3
- Sanga (statues, cultural, temples) | Budget: $500 | Days: 3

Day 4:
- Devghat (Religious, cultural, temples,river) | Budget: $500 | Days: 3
- Hanuman Dhoka (temples, cultural) | Budget: $500 | Days: 3

Day 5:
- Thimi (cultural, temples) | Budget: $500 | Days: 3
- Pata